In [1]:
#%%writefile get_proxy.py
#%load get_bookinfo.py
#%run get_bookinfo.py
# -*- coding: utf-8 -*-
#________________________________________________
import os
import django
from django.utils import timezone
#from django.utils.dateparse import parse_datetime
from datetime import datetime,date#,timezone
import pytz
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from difflib import SequenceMatcher
from IPython.display import clear_output, display
from time import sleep, time
from threading import Thread
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor 

from fake_useragent import UserAgent
from fake_headers import Headers
from pyquery import PyQuery as pq

#
import requests
import pandas as pd
import numpy as np
import random
import re
import json
import csv


def get_proxy(which='free',now=False):
    which_dict={'kuai':"/home/pan/django_projects/wtb/ips_kuai.txt",
                'ihuan':"/home/pan/django_projects/wtb/ips_1000_ihuan.txt",
                'free':"/home/pan/django_projects/wtb/ips_free.txt",
                'OK':"/home/pan/django_projects/wtb/ips_OKs.txt"
               }
    ippos=[]
    with open(which_dict[which], 'r') as f:
            lines=f.readlines()
            for line in lines:
                line=line.strip()
                if len(line.split(":"))==2:
                    ippos.append(line)
    if now:
        sleep(1.5)
        return random.choice(ippos)
    #ippo
    old=[]
    #測試代理_________________________________________
    while True:
        fake_header = Headers(
            browser="chrome",  # Generate only Chrome UA
            os="win",  # Generate ony Windows platform
            headers=True  # generate misc headers
        )    
        UA=fake_header.generate()
        #UA = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        #已經測過的就不要
        ippo=random.choice(ippos)
        if len(old)/len(ippos) > 0.5:
            old=[]
        if ippo in old:
            continue
        else:
            old.append(ippo)
        #    
        ip=ippo.split(":")[0]
        proxies={
                "http": "http://"+ippo,
                "https": "http://"+ippo
                }  
        #print("test:"+ippo)    
        #clear_output(wait=True)
        try:
            test=['http://icanhazip.com/','https://myip.com.tw/','https://www.showmyipaddress.eu/']
            r = requests.get(random.choice(test),#test[1], 
                             headers=UA,
                             proxies=proxies,
                             timeout=4)
            r.encoding='utf8'
            #print(r.status_code)
            #print(r.text)
            #return
            #return 22
            #
            if(r.status_code == 200 and (ip in r.text or '&#46;'.join(ip.split('.')) in r.text) ):
                print("OK:"+which+"_"+ippo)
                #print(r.status_code)
                #print(r.text)
                r.close() 
                if which!='OK':
                    with open('/home/pan/django_projects/wtb/ips_OKs.txt', 'a+') as f:
                        f.write(ippo+"\r\n")                
                return ippo
            else:#if not (r.status_code == 200 and ip in r.text):
                r.close()
                raise Exception('fail')
               
        except Exception as e:
            #print(str(e))
            #clear_output(wait=True) 
             
            sleep(1.5+random.uniform(0, 2))
            #sleep(0.1)
            continue
        #else:
        #    print(r.text)
        #    print("OK2:"+ip)
        #    r.close()       
        #    break    

In [ ]:
#kuai,ihuan,free
%time get_proxy('free')

In [2]:
if __name__ == '__main__':
    which_dict={
                #'kuai':"ips_kuai.txt",
                'ihuan':"ips_1000_ihuan.txt",
                'free':'ips_free.txt',
                #'OK':'ips_OKs.txt'
               }
    for i in range(20):
        print(i)
        with ThreadPoolExecutor(max_workers=3) as executor:
            executor.map(get_proxy,which_dict.keys())

0
OK:free_91.67.240.32:3128
OK:ihuan_27.191.234.69:9999
1
OK:free_181.196.205.250:44321
OK:ihuan_61.216.43.102:8080
2
OK:free_118.174.233.41:41469
OK:ihuan_114.31.15.62:8080
3
OK:free_173.46.67.172:58517
OK:ihuan_118.99.94.225:8080
4
OK:ihuan_39.137.95.70:80
OK:free_36.89.180.103:52092
5
OK:ihuan_45.32.119.44:8080
OK:free_128.199.188.231:47503
6
OK:free_45.113.192.180:8080
OK:ihuan_223.204.69.226:8080
7
OK:free_176.120.37.82:59365
OK:ihuan_189.240.124.61:8080
8
OK:free_60.248.199.206:80
OK:ihuan_51.158.99.51:8811
9
OK:free_128.199.188.231:47503
OK:ihuan_128.199.237.248:44344
10
OK:ihuan_163.172.136.226:8811
OK:free_201.49.72.226:8080
11
OK:free_36.89.181.161:50204
OK:ihuan_163.172.152.52:8811
12
OK:ihuan_213.183.99.110:8080
OK:free_150.129.52.74:6666
13
OK:ihuan_128.199.251.249:44344
OK:free_101.109.255.18:46545
14
OK:ihuan_36.67.116.165:8080
OK:free_213.129.99.10:61557
15
OK:free_181.196.151.82:49885
OK:ihuan_186.192.254.88:8080
16
OK:free_169.239.223.136:49027
OK:ihuan_170.84.83.186: